In [6]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 16: Proboscidea Volcanium ---</h2><p>The sensors have led you to the origin of the distress signal: yet another handheld device, just like the one the Elves gave you. However, you don't see any Elves around; instead, the device is surrounded by elephants! They must have gotten lost in these tunnels, and one of the elephants apparently figured out how to turn on the distress signal.</p>
<p>The ground rumbles again, much stronger this time. What kind of cave is this, exactly? You scan the cave with your handheld device; it reports mostly igneous rock, some ash, pockets of pressurized gas, magma... this isn't just a cave, it's a volcano!</p>
<p>You need to get the elephants out of here, quickly. Your device estimates that you have <em>30 minutes</em> before the volcano erupts, so you don't have time to go back out the way you came in.</p>
<p>You scan the cave for other options and discover a network of pipes and pressure-release <em>valves</em>. You aren't sure how such a system got into a volcano, but you don't have time to complain; your device produces a report (your puzzle input) of each valve's <em>flow rate</em> if it were opened (in pressure per minute) and the tunnels you could use to move between the valves.</p>
<p>There's even a valve in the room you and the elephants are currently standing in labeled <code>AA</code>. You estimate it will take you one minute to open a single valve and one minute to follow any tunnel from one valve to another. What is the most pressure you could release?</p>
<p>For example, suppose you had the following scan output:</p>
<pre><code>Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II
</code></pre>
<p>All of the valves begin <em>closed</em>. You start at valve <code>AA</code>, but it must be damaged or <span title="Wait, sir! The valve, sir! it appears to be... jammed!">jammed</span> or something: its flow rate is <code>0</code>, so there's no point in opening it. However, you could spend one minute moving to valve <code>BB</code> and another minute opening it; doing so would release pressure during the remaining <em>28 minutes</em> at a flow rate of <code>13</code>, a total eventual pressure release of <code>28 * 13 = <em>364</em></code>. Then, you could spend your third minute moving to valve <code>CC</code> and your fourth minute opening it, providing an additional <em>26 minutes</em> of eventual pressure release at a flow rate of <code>2</code>, or <code><em>52</em></code> total pressure released by valve <code>CC</code>.</p>
<p>Making your way through the tunnels like this, you could probably open many or all of the valves by the time 30 minutes have elapsed. However, you need to release as much pressure as possible, so you'll need to be methodical. Instead, consider this approach:</p>
<pre><code>== Minute 1 ==
No valves are open.
You move to valve DD.

== Minute 2 ==
No valves are open.
You open valve DD.

== Minute 3 ==
Valve DD is open, releasing <em>20</em> pressure.
You move to valve CC.

== Minute 4 ==
Valve DD is open, releasing <em>20</em> pressure.
You move to valve BB.

== Minute 5 ==
Valve DD is open, releasing <em>20</em> pressure.
You open valve BB.

== Minute 6 ==
Valves BB and DD are open, releasing <em>33</em> pressure.
You move to valve AA.

== Minute 7 ==
Valves BB and DD are open, releasing <em>33</em> pressure.
You move to valve II.

== Minute 8 ==
Valves BB and DD are open, releasing <em>33</em> pressure.
You move to valve JJ.

== Minute 9 ==
Valves BB and DD are open, releasing <em>33</em> pressure.
You open valve JJ.

== Minute 10 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve II.

== Minute 11 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve AA.

== Minute 12 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve DD.

== Minute 13 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve EE.

== Minute 14 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve FF.

== Minute 15 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve GG.

== Minute 16 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You move to valve HH.

== Minute 17 ==
Valves BB, DD, and JJ are open, releasing <em>54</em> pressure.
You open valve HH.

== Minute 18 ==
Valves BB, DD, HH, and JJ are open, releasing <em>76</em> pressure.
You move to valve GG.

== Minute 19 ==
Valves BB, DD, HH, and JJ are open, releasing <em>76</em> pressure.
You move to valve FF.

== Minute 20 ==
Valves BB, DD, HH, and JJ are open, releasing <em>76</em> pressure.
You move to valve EE.

== Minute 21 ==
Valves BB, DD, HH, and JJ are open, releasing <em>76</em> pressure.
You open valve EE.

== Minute 22 ==
Valves BB, DD, EE, HH, and JJ are open, releasing <em>79</em> pressure.
You move to valve DD.

== Minute 23 ==
Valves BB, DD, EE, HH, and JJ are open, releasing <em>79</em> pressure.
You move to valve CC.

== Minute 24 ==
Valves BB, DD, EE, HH, and JJ are open, releasing <em>79</em> pressure.
You open valve CC.

== Minute 25 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

== Minute 26 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

== Minute 27 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

== Minute 28 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

== Minute 29 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

== Minute 30 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.
</code></pre>

<p>This approach lets you release the most pressure possible in 30 minutes with this valve layout, <code><em>1651</em></code>.</p>
<p>Work out the steps to release the most pressure in 30 minutes. <em>What is the most pressure you can release?</em></p>
</article>


In [7]:
from collections import deque

# Minute pressure    opening open Valves
#  1            0
#  2            0   DD                          20*28 = 560
#  3           20            DD
#  4           20            DD
#  5           20   BB       DD                 560+25*13 = 885
#  6           33            DD,BB
#  7           33            DD,BB
#  8           33            DD,BB
#  9           33   JJ       DD,BB              885+21*21 = 1326
# 10           54            DD,BB,JJ
# 11           54            DD,BB,JJ
# 12           54            DD,BB,JJ
# 13           54            DD,BB,JJ
# 14           54            DD,BB,JJ
# 15           54            DD,BB,JJ
# 16           54            DD,BB,JJ
# 17           54    HH      DD,BB,JJ           1326+13*22=1612
# 18           76            DD,BB,JJ,HH
# 19           76            DD,BB,JJ,HH
# 20           76            DD,BB,JJ,HH
# 21           76    EE      DD,BB,JJ,HH        1612+9*3= 1639
# 22           79            DD,BB,JJ,HH.EE
# 23           79            DD,BB,JJ,HH.EE
# 24           79    CC      DD,BB,JJ,HH.EE     1639+6*2= 1651
# 25           81            DD,BB,JJ,HH.EE,CC
# 26           81            DD,BB,JJ,HH.EE,CC
# 27           81            DD,BB,JJ,HH.EE,CC
# 28           81            DD,BB,JJ,HH.EE,CC
# 29           81            DD,BB,JJ,HH.EE,CC
# 30           81            DD,BB,JJ

tests = [
    {
        "name": "Example",
        "s": """
            Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
            Valve BB has flow rate=13; tunnels lead to valves CC, AA
            Valve CC has flow rate=2; tunnels lead to valves DD, BB
            Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
            Valve EE has flow rate=3; tunnels lead to valves FF, DD
            Valve FF has flow rate=0; tunnels lead to valves EE, GG
            Valve GG has flow rate=0; tunnels lead to valves FF, HH
            Valve HH has flow rate=22; tunnel leads to valve GG
            Valve II has flow rate=0; tunnels lead to valves AA, JJ
            Valve JJ has flow rate=21; tunnel leads to valve II
        """,
        "expected": 1651,
    },
]


class Network:
    def __init__(self, s: str) -> None:
        data = [
            re.findall(r"([A-Z]{2}|\d+)", l.strip()) for l in s.strip().splitlines()
        ]
        self.G = dict(sorted((n, (int(r), c)) for n, r, *c in data))

    def most_pressure(self, t: int = 30) -> int:
        nodes, node_to_idx, distances = self.floyd_warshall()

        valves = [n for n in nodes if self.G[n][0]]

        min_distance = min(
            distances[node_to_idx[v1]][node_to_idx[v2]]
            for v1, v2 in product(valves, repeat=2)
            if v1 != v2
        )

        stack = deque([(0, 0, t, "AA", valves)])

        max_pressure = -inf

        while stack:
            acc, pressure, time, current, valves = stack.pop()

            if time < 0:
                continue

            max_pressure = max(max_pressure, acc + pressure * time)

            if time < min_distance:
                continue

            for i in range(len(valves)):
                dt = distances[node_to_idx[current]][node_to_idx[valves[i]]] + 1
                stack.append(
                    (
                        acc + dt * pressure,
                        pressure + self.G[valves[i]][0],
                        time - dt,
                        valves[i],
                        valves[:i] + valves[i + 1 :],
                    )
                )

        return max_pressure

    def print_distances(self, nodes, distances) -> None:
        print(
            "\n".join(
                (
                    f"    {'  '.join(f'{n}' for n in nodes)}",
                    "\n".join(
                        f'{nodes[r]} {" ".join(f"{i:3}" for i in row)}'
                        for r, row in enumerate(distances)
                    ),
                ),
            )
        )

    def floyd_warshall(self) -> tuple[list[str], dict[str, int], list[list[int]]]:
        nodes = list(self.G.keys())
        node_to_idx = {n: i for i, n in enumerate(nodes)}
        n = len(nodes)

        distances = [[inf] * n for _ in range(n)]

        for node, (_, children) in self.G.items():
            for child in children:
                distances[node_to_idx[node]][node_to_idx[child]] = 1

        for i in range(n):
            distances[i][i] = 0

        for k, i, j in product(range(n), repeat=3):
            distances[i][j] = min(distances[i][j], distances[i][k] + distances[k][j])

        return nodes, node_to_idx, distances

    def show(self, node_size: int = 1_400) -> Network:
        G = nx.Graph({n: c for n, (_, c) in self.G.items()})

        node_labels = {n: f"{n}:{r}" for n, (r, _) in self.G.items()}
        nx.set_node_attributes(G, node_labels, name="label")

        pos = nx.spring_layout(G, pos={"AA": (0, 0)}, fixed=["AA"], seed=42)
        nx.draw(G, pos, node_color="skyblue", node_size=node_size, arrows=False)
        nx.draw_networkx_labels(G, pos, labels=nx.get_node_attributes(G, "label"))
        plt.show()
        return self

    def __str__(self) -> str:
        return "\n".join(
            f"{ n}, rate={r} and children={c}" for n, (r, c) in self.G.items()
        )


@test(tests=tests)
def test_part_I(s: str) -> int:
    return Network(s).most_pressure()


Test Example passed, for test_part_I.
Success


In [8]:
with open("../input/day16.txt") as f:
    puzzle = f.read()

print(f"Part II: {Network(puzzle).most_pressure()}")

Part II: 1754


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1754</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You're worried that even with an optimal approach, the pressure released won't be enough. What if you got one of the elephants to help you?</p>
<p>It would take you 4 minutes to teach an elephant how to open the right valves in the right order, leaving you with only <em>26 minutes</em> to actually execute your plan. Would having two of you working together be better, even if it means having less time? (Assume that you teach the elephant before opening any valves yourself, giving you both the same full 26 minutes.)</p>
<p>In the example above, you could teach the elephant to help you as follows:</p>
<pre><code>== Minute 1 ==
No valves are open.
You move to valve II.
The elephant moves to valve DD.

== Minute 2 ==
No valves are open.
You move to valve JJ.
The elephant opens valve DD.

== Minute 3 ==
Valve DD is open, releasing <em>20</em> pressure.
You open valve JJ.
The elephant moves to valve EE.

== Minute 4 ==
Valves DD and JJ are open, releasing <em>41</em> pressure.
You move to valve II.
The elephant moves to valve FF.

== Minute 5 ==
Valves DD and JJ are open, releasing <em>41</em> pressure.
You move to valve AA.
The elephant moves to valve GG.

== Minute 6 ==
Valves DD and JJ are open, releasing <em>41</em> pressure.
You move to valve BB.
The elephant moves to valve HH.

== Minute 7 ==
Valves DD and JJ are open, releasing <em>41</em> pressure.
You open valve BB.
The elephant opens valve HH.

== Minute 8 ==
Valves BB, DD, HH, and JJ are open, releasing <em>76</em> pressure.
You move to valve CC.
The elephant moves to valve GG.

== Minute 9 ==
Valves BB, DD, HH, and JJ are open, releasing <em>76</em> pressure.
You open valve CC.
The elephant moves to valve FF.

== Minute 10 ==
Valves BB, CC, DD, HH, and JJ are open, releasing <em>78</em> pressure.
The elephant moves to valve EE.

== Minute 11 ==
Valves BB, CC, DD, HH, and JJ are open, releasing <em>78</em> pressure.
The elephant opens valve EE.

(At this point, all valves are open.)

== Minute 12 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

...

== Minute 20 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.

...

== Minute 26 ==
Valves BB, CC, DD, EE, HH, and JJ are open, releasing <em>81</em> pressure.
</code></pre>

<p>With the elephant helping, after 26 minutes, the best you could do would release a total of <code><em>1707</em></code> pressure.</p>
<p><em>With you and an elephant working together for 26 minutes, what is the most pressure you could release?</em></p>
</article>


In [9]:
from collections import deque
from itertools import combinations
from math import factorial
from pprint import pprint

from more_itertools import minmax

# Minute pressure    opening  open Valves
#                    me ele
#  1             0
#  2             0        DD                        (26-2)*20=               480
#  3             20   JJ       DD                   (26-3)*21+480=           963
#  4             41            DD,JJ
#  5             41            DD,JJ
#  6             41            DD,JJ
#  7             41    BB  HH  DD,JJ                (26-7)*(13+22)+963=     1628
#  8             76            DD,JJ,BB,HH
#  9             76    CC      DD,JJ,BB,HH          (26-9)*2+1628=          1662
# 10             78            DD,JJ,BB,HH.CC
# 11             78        EE  DD,JJ,BB,HH.CC       (26-11)*3+1662=         1707
# 12             81            DD,JJ,BB,HH.CC,EE


tests = [
    {
        "name": "Example",
        "s": """
            Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
            Valve BB has flow rate=13; tunnels lead to valves CC, AA
            Valve CC has flow rate=2; tunnels lead to valves DD, BB
            Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
            Valve EE has flow rate=3; tunnels lead to valves FF, DD
            Valve FF has flow rate=0; tunnels lead to valves EE, GG
            Valve GG has flow rate=0; tunnels lead to valves FF, HH
            Valve HH has flow rate=22; tunnel leads to valve GG
            Valve II has flow rate=0; tunnels lead to valves AA, JJ
            Valve JJ has flow rate=21; tunnel leads to valve II
        """,
        "t": 26,
        "expected": 1707,
    },
]


class NetworkII(Network):
    def most_pressure_with_elefant(self, t: int = 26) -> int:
        distances = self.reduced_distances_matrrix()
        flows = [f for f, _ in self.G.values() if f]
        masks = [1 << i for i in range(len(flows))]

        totals = defaultdict(int)
        stack = deque([(0, t, 0, 0)])

        while stack:
            current, time, mask, pressure = stack.pop()

            totals[mask] = max(totals[mask], pressure)

            for nxt, flow in enumerate(flows):
                t_rest = time - distances[current][nxt + 1] - 1

                if not masks[nxt] & mask and t_rest > 0:
                    stack.append(
                        (
                            nxt + 1,
                            t_rest,
                            mask | masks[nxt],
                            pressure + flow * t_rest,
                        )
                    )

        return max(
            v1 + v2
            for (m1, v1), (m2, v2) in product(totals.items(), repeat=2)
            if not m1 & m2
        )

    def reduced_distances_matrrix(self) -> list[list[int]]:
        nodes, node_to_idx, distances = self.floyd_warshall()
        valves = ["AA"] + [n for n in self.G if self.G[n][0]]
        impportant_valves = [node_to_idx[n] for n in valves]
        d = 0

        for i in range(len(distances)):
            if not i in impportant_valves:
                nodes.pop(i - d)
                distances.pop(i - d)
                distances = [list(r) for r in zip(*distances)]
                distances.pop(i - d)
                distances = [list(r) for r in zip(*distances)]
                d += 1

        return distances


@test(tests=tests[:1])
def test_part_II(s: str, t: int) -> int:
    return NetworkII(s).most_pressure_with_elefant(t)


Test Example passed, for test_part_II.
Success


In [10]:
print(f"Part II: {NetworkII(puzzle).most_pressure_with_elefant()}")

Part II: 2474


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>2474</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
